<div class="alert alert-block alert-success">
<h1><strong>NASA Data Fusion Analysis of Derechos and Their Impact on Rural America</h1>
</div>
    
Story authors: Madison Wallner<sup>1</sup>, Andrew Blackford<sup>1</sup>, Udaysankar Nair<sup>1</sup><br />
<sup>1</sup>The University of Alabama in Huntsville (UAH)

Notebook editor: Kyle Lesinger<sup>1</sup>

# Run This Notebook

<div class="alert alert-block alert-warning">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes packages specific to VEDA stories, such as maap-py. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed.</div>

## Example scripts to process and visualize information

In [1]:
!pip install plotly
# Load libraries
import videoutils as vutils
import readutils as rutils
import plotutils as putils
import vedautils as veda
import statsutils as sutils
import earthaccess
import datetime
import pandas as pd
import xarray as xr
import shutil


NameError: name 't' is not defined

In [ ]:
#For retrieving data already catalogued in VEDA STAC
STAC_API_URL="https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"


# What is a Derecho?

A **derecho** is a long-lasting, fast-moving windstorm that is produced from a line of severe thunderstorms. For a storm to be called a derecho, there must be:

- A concentrated area of severe wind reports over 400 miles (650 km) in length and at least 60 miles (96 km) wide.
- Several wind gusts reported over 75 mph (120 km/h).
- Sustained winds of at least 58 mph (93 km/h). (SPC)

Unlike single thunderstorms, derechos form within mesoscale (i.e., mid-sized) convective systems (MCS)—large storm clusters that produce strong, long-lasting straight-line winds.

## Conditions that Help a Derecho Form Include:

- **Strong instability** (Convective Available Potential Energy (CAPE) over 2000–4000 J/kg): provides energy for strong updrafts and intense thunderstorms.  
- **High low-level moisture** (dewpoints of 65–75 °F): keeps storms going by supplying moisture.  
- **Strong mid- and upper-level winds** (wind shear over 40 knots): help organize storms and push them forward.  
- **A well-defined cold pool**: rain-cooled air at the surface strengthens the storm by increasing wind speeds at the front of the system.  


<div class="alert alert-block alert-warning">
<h1><strong>Example:</strong> Retrieve GLDAS data from EarthData portal</h1>
<h3>Global Land Data Assimilation System (GLDAS) data helps assess soil moisture levels before and after the storm, showing how pre-existing drought conditions contributed to dust transport and how heavy rainfall may have impacted runoff and flooding.</h3>
</div>


### Processing steps:
1.) Select a start and end date for the data granules<br />
2.) Add <a href="https://search.earthdata.nasa.gov/search/granules?p=C1690022359-GES_DISC&pg[0][v]=f&pg[0][gsk]=-start_date&q=GLDAS_CLSM10_3H_2.1&tl=1347014600.484!5!!&lat=0.05719976537514171&long=-0.07464366545408296&zoom=3.0033333333333334"
        target="_blank" rel="noopener noreferrer"
         style="color: blue; text-decoration: none;">
      bucket name from EarthData portal
    </a><br />
3.) Add the prefix for the data within the bucket (e.g., remote sensing mission, reanalysis model, etc.)<br />
4.) Retrieve all files within the collection based on the start and end date. (If the same year is selected for start_date and end_date, then only the common year will be retrieved. Else all granules are retrived.<br />
5.) Filter all collected granules based on the start_ and end_date range.<br />
6.) Compute daily mean of hourly files<br />
7.) Plot variable based on coordinates selected<br />
8.) Create a .gif over the start_ and end_date range

### Select dates and find Earthdata links

In [ ]:
#Only subset data within a single year (requires >3GB RAM if you select more than 40 dates)
start_date = datetime.datetime(2022, 4, 1)
end_date = datetime.datetime(2022, 5, 12)
date_array = pd.date_range(start=start_date, end=end_date, freq='D').to_pydatetime()

bucket_name = "gesdisc-cumulus-prod-protected"
prefix = "GLDAS/GLDAS_NOAH025_3H.2.1"

#Don't get all of the data links if we don't have too. Only chooses a single year if the start_ and end_dates are in the same year
prefix = f'{prefix}/{start_date.year}' if (start_date.year) == (end_date.year) else prefix

# #Download all links for either a specific year or for all years
all_links = rutils.list_files(bucket_name=bucket_name,
                     prefix = prefix,
                     region = "us-west-2",
                     file_extension = ".nc4")


### Filter based on start_date and end_date

In [ ]:
#Subset the data based on the actual start_date and end_date to reduce number of processes
filtered_keys = []

for key in all_links:
    # Example: extract "20220401" from filename
    date_str = key.split('.')[-4][1:]  # A20220401 -> 20220401
    file_date = datetime.datetime.strptime(date_str, "%Y%m%d")
    
    if start_date <= file_date <= end_date:
        filtered_keys.append(key)

print(f"Filtered to {len(filtered_keys)} files between {start_date.strftime("%B")} {start_date.day} and {end_date.strftime("%B")} {end_date.day}, {end_date.year}.")

### Compute daily mean (soil moisture data is within 3-hr increments)

In [ ]:
variable = 'SoilMoi0_10cm_inst'

'''Pass filtered keys through EarthData portal, stores in a dictionary, 
and computes the daily mean'''
daily_mean_datasets = sutils.compute_daily_mean_from_keys_NETCDF(
    bucket_name = bucket_name,
    region_name = "us-west-2",
    file_keys=filtered_keys,
    variable = variable
)
print(list(daily_mean_datasets.keys())[0])

### Convert to a single xarray object

In [ ]:
#Select a soil moisture variable (0-10cm); items are currently in a dictionary
soil = [daily_mean_datasets[i] for i in list(daily_mean_datasets.keys())]
soil = xr.concat(soil,dim='time')
soil['time'] = date_array
print(f"First 10 dates: {soil.time.values[0:10]}")


### Create interactive geospatial folium plot

In [ ]:
putils.plot_folium_from_xarray(dataset=soil,
                  day_select='2022-05-11',
                  bbox=[-130, 33, -90, 50],
                  var_name_for_title='Soil Moisture from GLDAS (m³/m³) [subset]',
                  matplot_ramp = 'YlOrRd_r',
                  zoom_level = 5,
                  flipud=False,
                  save_tif=False,
                  tif_filename=None,
                  crs = 'EPSG4326',
                  opacity = 0.8)


### Create a gif over the date range

In [ ]:
# Usage:
putils.matplotlib_gif(data=soil,
               bbox= [-104, 30, -90, 50],
               gif_savename= "soil_matplotlib.gif",
               duration=2,
               cmap="YlOrRd_r")

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull WLDAS soil moisture data from the VEDA STAC catalog and visualize.</h2>

The Western Land Data Assimilation System (WLDAS) is a regional instance of NASA’s Land Information System (LIS), developed at Goddard Space Flight Center for the western United States.  It integrates meteorological forcings (precipitation, radiation, temperature, humidity, wind, surface pressure) and satellite-derived parameters (vegetation class, soil texture, elevation) into the Noah-MP land surface model using data assimilation techniques.

Soil moisture critically controls the partitioning of net surface energy into latent (evapotranspiration) versus sensible (heating) fluxes. Wetter soils favor latent heat, stabilizing the boundary layer, whereas drier soils boost sensible heating, enhancing near-surface temperature and convective available potential energy (CAPE). These processes govern where and when thunderstorms can initiate and organize.
</div>


### Processing steps:
1.) Choose STAC catalog ID and date<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

### Choose variable and retrieve json

In [ ]:
collection_soil = 'wldas-derecho-sm'
collection = veda.get_collection(STAC_API_URL, collection_soil)
print(f'Collection information: {collection}')

items = veda.search_stac_features(stac_api_url=STAC_API_URL,
                               collection_id=collection_soil,
                               date="2022-05-11",
                               limit=100)

print(f"\nNumber of collection items found: {len(items)}")
item = items[0] #only return the first item

#Collection render information to display like the VEDA story
dashboard_rend, bands, asset_keys, vmin, vmax, cmap_name = veda.return_render_information(collection)


### Retrieve tiling information

In [ ]:
tiles = veda.get_raster_tilejson(raster_api_url = RASTER_API_URL,
                                 collection_id = collection_soil,
                                 item = item,
                                 rescale = (vmin,vmax),
                                 assets = asset_keys[0],
                                 color_formula = "gamma+r+1.05",
                                 colormap_name = cmap_name
                                )
                                 


tiles

### Plot data

In [ ]:
# Example usage:
tiles_url = tiles["tiles"][0]  # from your TileJSON
lat_min, lon_min, lat_max, lon_max = 30, -110, 50, -95
center = ((lat_min + lat_max) / 2, (lon_min + lon_max) / 2)

m = putils.plot_folium_from_STAC_with_legend(
    tiles_url=tiles_url,
    vmin = vmin,
    vmax = vmax,
    center=center,
    zoom_start=5.5,
    width="100%",
    height="800px",
    attribution="VEDA",
    layer_name="WLDAS soil moisture",
    day_select = item['properties'].get('start_datetime'),
    crs= "EPSG3857",
    opacity = 0.6,
    colormap_name = "".join(
    c.upper() if i % 2 == 0 else c.lower()
    for i, c in enumerate(cmap_name)
)
)

# In a Jupyter notebook, simply display `m`:
m

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Retrieve MERRA-2 hourly files for Aerosol Optical Thickness</h2>

Aerosol Optical Thickness (AOT), also called Aerosol Optical Depth (AOD), is a dimensionless measure of how much sunlight aerosols—tiny particles like dust, smoke or sea salt—scatter and absorb as it travels through a column of atmosphere. In practical terms, an AOT of 0.1 means only 10 % of the direct solar beam is extinguished by aerosols before reaching the surface.

The intense straight-line winds in derechos can uplift large quantities of soil and dust, dramatically increasing AOT downwind. Tracking AOT in near-real-time reveals the spatial extent and intensity of these airborne dust plumes.
</div>


### Processing steps:
1.) Select a start and end date for the data granules<br />
2.) Add the bucket name from EarthData portal<br />
3.) Add the prefix for the data within the bucket (e.g., remote sensing mission, reanalysis model, etc.)<br />
4.) Retrive all files within the collection based on the start and end date. (If the same year is selected for start_date and end_date, then only the common year will be retrieved. Else all granules are retrived.<br />
5.) Filter all collected granules based on the start_ and end_date range.<br />
6.) Plot variable based on coordinates selected<br />
7.) Create a .gif over the start_ and end_date range

### Select and filter

In [ ]:
#Only subset data within a single year
start_date = datetime.datetime(2022, 5, 12)
end_date = datetime.datetime(2022, 5, 12)
date_array = pd.date_range(start=start_date, end=end_date, freq='D').to_pydatetime()

bucket_name = "gesdisc-cumulus-prod-protected"
prefix = "MERRA2/M2T1NXAER.5.12.4"

#Don't get all of the data links if we don't have too. Only chooses a single year if the start_ and end_dates are in the same year
prefix = f'{prefix}/{start_date.year}' if (start_date.year) == (end_date.year) else prefix

#Download all links for either a specific year or for all years
all_links = rutils.list_files(bucket_name=bucket_name,
                     prefix = prefix,
                     region = "us-west-2",
                     file_extension = ".nc4")


#Subset the data based on the start_date and end_date to reduce number of processes
filtered_keys = []

for key in all_links:
    # Example: extract "20220401" from filename
    date_str = key.split('.')[-2]  # A20220401 -> 20220401
    file_date = datetime.datetime.strptime(date_str, "%Y%m%d")
    
    if start_date <= file_date <= end_date:
        filtered_keys.append(key)

print(f"Filtered to {len(filtered_keys)} files between {start_date.strftime("%B")} {start_date.day} and {end_date.strftime("%B")} {end_date.day}, {end_date.year}.")



In [ ]:
variable = 'TOTEXTTAU' #Total aerosol extinction optical thickness

# Compute daily mean for all variables within each file
# Assume `filtered_keys` is your list of S3 paths (.nc4 files)
merra_datasets = rutils.load_datasets_from_keys(
    bucket_name = bucket_name,
    region_name = "us-west-2",
    file_keys=filtered_keys,
    variable = variable,
    data_source = 'MERRA'
)
print(list(merra_datasets.keys())[0])


In [ ]:
# Currently each day is its own key
merra_datasets

In [ ]:
#Only grab the first object in the dictionary
single_date = merra_datasets.get(list(merra_datasets.keys())[0])[0]

### Plot

In [ ]:
# Usage:
putils.matplotlib_gif(data=single_date[variable],
               bbox= [-110, 30, -90, 40],
               gif_savename= "merra_aot_matplotlib.gif",
               duration=2,
               cmap="YlOrRd_r")


## Retrieve and plot MODIS Aerosol Optical Depth

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull MODIS Aerosol Optical Depth data from the VEDA STAC catalog and visualize</h2>
</div>


In [ ]:
collection_modis = 'modis-derecho'
date = "2022-05-12"

collection = veda.get_collection(STAC_API_URL, collection_modis)
collection

items = veda.search_stac_features(stac_api_url=STAC_API_URL,
                               collection_id=collection_modis,
                               date=date,
                               limit=100)

print(f"Number of collection items found: {len(items)}")
item = items[0] #only return the first item

In [ ]:
#Collection render information to display like the VEDA story
dashboard_rend, bands, asset_keys, vmin, vmax, cmap_name = veda.return_render_information(collection)


tiles = veda.get_raster_tilejson(raster_api_url = RASTER_API_URL,
                                 collection_id = collection_modis,
                                 item = item,
                                 rescale = (vmin, vmax),
                                 assets = asset_keys[0],
                                 color_formula = "gamma+r+1.05",
                                 colormap_name = cmap_name
                                )
                                 


tiles

In [ ]:
# Example usage:
tiles_url = tiles["tiles"][0]  # from your TileJSON
lat_min, lon_min, lat_max, lon_max = 35, -100, 50, -85
center = ((lat_min + lat_max) / 2, (lon_min + lon_max) / 2)

m = putils.plot_folium_from_STAC_with_legend(
    tiles_url=tiles_url,
    vmin = vmin,
    vmax = vmax,
    center=center,
    zoom_start=7,
    width="100%",
    height="800px",
    attribution="VEDA",
    layer_name="MODIS AOD",
    day_select = item['properties'].get('datetime'),
    crs= "EPSG3857",
    opacity = 0.7,
    colormap_name = "".join(
    c.upper() if i % 2 == 0 else c.lower()
    for i, c in enumerate(cmap_name)
)
)

# In a Jupyter notebook, simply display `m`:
m

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull NCEI interpolated wind gusts data from the VEDA STAC catalog and visualize.</h2>
    
The NCEI Interpolated Wind Gusts product takes the discrete station‐reported peak wind gusts from the NCEI Storm Events Database—a standardized archive of severe‐weather observations dating back to 1950—and uses spatial interpolation to generate a continuous gridded field of maximum gust speeds across the derecho swath. 

By filling in the gaps between point measurements, it reveals the full geographic extent and intensity gradients of the derecho’s outflow winds, often uncovering zones of extreme gust (e.g., 70 mph +) that lie between and beyond individual station sites. 

</div>


In [ ]:
collection_wind = 'windgusts-derecho'
date = "2022-05-12"
collection = veda.get_collection(STAC_API_URL, collection_wind)
collection

items = veda.search_stac_features(stac_api_url=STAC_API_URL,
                               collection_id=collection_wind,
                               date=date,
                               limit=100)

print(f"Number of collection items found: {len(items)}")
item = items[0] #only return the first item

In [ ]:
#Collection render information to display like the VEDA story
dashboard_rend, bands, asset_keys, vmin, vmax, cmap_name = veda.return_render_information(collection)


tiles = veda.get_raster_tilejson(raster_api_url = RASTER_API_URL,
                                 collection_id = collection_wind,
                                 item = item,
                                 rescale = (vmin, vmax),
                                 assets = asset_keys[0],
                                 color_formula = "gamma+r+1.05",
                                 colormap_name = cmap_name
                                )
                                 


tiles

In [ ]:
# Example usage:
tiles_url = tiles["tiles"][0]  # from your TileJSON
lat_min, lon_min, lat_max, lon_max = 35, -105, 50, -90
center = 43.8250, -97.7

m = putils.plot_folium_from_STAC_with_legend(
    tiles_url=tiles_url,
    vmin = vmin,
    vmax = vmax,
    center=center,
    zoom_start=7,
    width="100%",
    height="800px",
    attribution="VEDA",
    layer_name="Windgusts",
    day_select = item['properties'].get('datetime'),
    crs= "EPSG3857",
    opacity = 0.7,
    colormap_name = "".join(
    c.upper() if i % 2 == 0 else c.lower()
    for i, c in enumerate(cmap_name)
)
)

# In a Jupyter notebook, simply display `m`:
m

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull Black Marble Night Lights data from the VEDA STAC catalog and visualize.</h2>

During extreme windstorms such as derechos, sudden drops in nighttime brightness in the Black Marble imagery reveal where power infrastructure has failed and outages have occurred, while the re-illumination of areas over subsequent days tracks the pace and extent of electrical service restoration. This makes Black Marble a powerful, near-real-time proxy for assessing societal impacts and grid resilience in the storm’s wake.
</div>


In [ ]:
collection_nightlights = 'nightlights-derecho'
date = "2022-05-10"

collection = veda.get_collection(STAC_API_URL, collection_nightlights)
collection

items = veda.search_stac_features(stac_api_url=STAC_API_URL,
                               collection_id=collection_nightlights,
                               date=date,
                               limit=100)

print(f"Number of collection items found: {len(items)}")
item = items[0] #only return the first item

In [ ]:
#Collection render information to display like the VEDA story
dashboard_rend, bands, asset_keys, vmin, vmax, cmap_name = veda.return_render_information(collection)


tiles = veda.get_raster_tilejson(raster_api_url = RASTER_API_URL,
                                 collection_id = collection_nightlights,
                                 item = item,
                                 rescale = (vmin,vmax),
                                 assets = asset_keys[0],
                                 color_formula = "gamma+r+1.05",
                                 colormap_name = cmap_name
                                )
                                 


tiles

In [ ]:
# Example usage:
tiles_url = tiles["tiles"][0]  # from your TileJSON
lat_min, lon_min, lat_max, lon_max = 35, -105, 50, -90
center = 43.55,-96.94

m = putils.plot_folium_from_STAC_with_legend(
    tiles_url=tiles_url,
    vmin = vmin,
    vmax = vmax,
    center=center,
    zoom_start=10,
    width="100%",
    height="800px",
    attribution="VEDA",
    layer_name="Black Marble Night Lights",
    day_select = item['properties'].get('datetime'),
    crs= "EPSG3857",
    colormap_name = cmap_name,
    opacity = 0.7
)

# In a Jupyter notebook, simply display `m`:
m

## The economic impact reached beyond the crop fields. Many grain silos, irrigation systems, and farm buildings were also damaged or destroyed, adding to the financial burden on farmers.  See image below to identify crop types across the Midwest and the Storm Prediction Center reports across the Midwest.

### Use the slider below to investigate the crop-land type and the extreme weather reports across the Midwest.

In [ ]:
putils.plotly_dual_slider_window(image1_path="images/derecho/Derecho-Crop.jpg",
                                    image2_path="images/derecho/Derecho-storm-reports.jpg",
                                    target_width=800,
                                    fig_width=800,
                                    fig_height=600)

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull Global Precipitation Measurement data through the Common Metadata Repository (CMR) STAC API.</h2>

NASA's Global Precipitation Measurement Mission (GPM) uses satellites to measure Earth's rain and snowfall for the benefit of mankind. Launched by NASA and JAXA on Feb. 27th, 2014, GPM is an international mission that sets the standard for spaceborne precipitation measurements.
</div>


In [ ]:
# GENERATED USING https://dev-titiler-cmr.delta-backend.com/api.html#/TileJSON/tilejson_endpoint__tileMatrixSetId__tilejson_json_get

#Specific for May 05, 2022
tilejson_url = 'https://dev-titiler-cmr.delta-backend.com/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?concept_id=C2723754864-GES_DISC&datetime=2022-05-12T09%3A00%3A00Z&variable=precipitation&backend=xarray&colormap_name=blues&rescale=0%2C48'

In [ ]:
center= 43.55,-96.94

m = putils.plot_folium_from_STAC_with_legend(
    tiles_url=tilejson_url,
    vmin = 0,
    vmax = 46,
    center= center,
    zoom_start=4,
    width="100%",
    height="800px",
    attribution="VEDA",
    layer_name="GPM Imerge Precipitation",
    day_select = "2022-05-12",
    crs= "EPSG3857",
    colormap_name = "Blues",
    opacity = 0.6,
)

m


<div class="alert alert-block alert-danger">
<h2><strong>End of visualizations</strong>
</div>


In [ ]:
# Remove created .gifs
import os
import glob

# find all .gif files in the current directory
for gif_path in glob.glob("*.gif"):
    try:
        os.remove(gif_path)
        print(f"Removed {gif_path}")
    except OSError as e:
        print(f"Error removing {gif_path}: {e}")
